构建RAG的第一步是构建私域数据的索引，以便后续提取到相关上下文。构建索引一般分为以下三个步骤：

![alt text](../../doc/image.png)

- **加载数据**：从不同的数据源加载数据，如txt、pdf、excle、sql等
- **转换数据**：加载上来的数据不直接存储，需要分块、提取元数据等操作
- **索引及存储数据**：构建分块的索引，存储到向量库中

## 加载数据

加载数据是为了从Markdown、PDF、Word 文档、PowerPoint 幻灯片、图像、音频和视频等不同数据源提取数据，并将其用于初始化Document对象

最简单的加载器是SimpleDirectoryReader，从指定目录加载Markdown、PDF、Word 文档、PowerPoint 幻灯片、图像、音频和视频等数据源

In [1]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("data\三国演义白话文",recursive=True).load_data(show_progress=True)
print(len(documents))

Loading files: 100%|██████████████| 42/42 [00:00<00:00, 44.97file/s]

43


除此之外，我们可以使用更加低级的接口，直接将文本加载为Document对象

In [4]:
from llama_index.core import Document

doc1=Document(text="123456")
doc2=Document(text="abcdef")

documents=[doc1,doc2]
print(documents)

[Document(id_='dfb3fd37-f8a6-4203-96a4-9509b28537ee', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='123456', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'), Document(id_='75fec794-dabd-47ca-b232-47954c153b41', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='abcdef', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}')]


除了以上LlamaIndex直接提供的接口外，在[Llama Hub](https://llamahub.ai/?tab=readers) 提供针对不同数据源的加载方式，还提供将这些加载对象转为其他框架的接口，以下是针对网页的加载器示例

In [9]:
# 默认不带这个加载器，通过命令安装：python -m pip install llama-index-readers-web
from llama_index.readers.web import SimpleWebPageReader

# 从网页加载数据
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://docs.llamaindex.ai/en/stable/understanding/loading/loading/"]
)
print(len(documents))

# 加载模型 
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
Settings.llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url='http://192.168.3.155:11434')

# 创建摘要索引
from llama_index.core import SummaryIndex
from IPython.display import Markdown, display
index = SummaryIndex.from_documents(documents)

# 创建查询引擎并查询内容
query_engine = index.as_query_engine()
response = query_engine.query("LlamaIndex加载数据有几种方式？")

# 显示查询结果
display(Markdown(f"<b>{response}</b>"))

1


<b>LlamaIndex 加载数据主要有以下几种方式：

1. **使用 `SimpleDirectoryReader` 载入简单目录中的文件**：
   ```python
   from llama_index.core import SimpleDirectoryReader
   
   documents = SimpleDirectoryReader("./data").load_data()
   ```

2. **利用读取器从 LlamaHub 加载数据**：
   例如，使用 `DatabaseReader` 从 SQL 数据库加载数据。
   ```python
   from llama_index.core import download_loader
   
   from llama_index.readers.database import DatabaseReader
   
   reader = DatabaseReader(
       scheme=os.getenv("DB_SCHEME"),
       host=os.getenv("DB_HOST"),
       port=os.getenv("DB_PORT"),
       user=os.getenv("DB_USER"),
       password=os.getenv("DB_PASS"),
       dbname=os.getenv("DB_NAME"),
   )
   
   query = "SELECT * FROM users"
   documents = reader.load_data(query=query)
   ```

3. **直接创建文档**：
   ```python
   from llama_index.core import Document
   
   doc = Document(text="text")
   ```

这些方法提供了灵活的数据加载选项，可以根据具体需求选择合适的方式来处理和加载数据。</b>

## 转换

转换的目的是将documents内的所有文档进行分块、提取元数据操作，在代码层面，就是将documents转为分块的nodes

以下代码通过自定义文本分割器，实现文本切割

In [19]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import TokenTextSplitter

documents = SimpleDirectoryReader("data\三国演义白话文",recursive=True).load_data(show_progress=True)
print(len(documents))

pipeline=IngestionPipeline(transformations=[
    TokenTextSplitter(chunk_size=500,chunk_overlap=100)])

nodes=pipeline.run(documents=documents)
print(len(nodes),'\n',nodes[0])


Loading files: 100%|████████████| 41/41 [00:00<00:00, 1320.46file/s]


41
258 
 Node ID: 371f3091-6281-4e0f-a771-04ca7368afe4
Text: 第一回 桃园结义     　　东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与他的弟弟张
梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起向官兵进攻。 　　没有几天，四方百姓，头裹黄巾，跟随张角三兄弟杀向
官府，声势非常浩大。汉灵帝得到各地报告，连忙下令各地官军防备。又派中郎将卢植、皇甫嵩、朱隽率领一精一兵，分路攻打张角兄弟的黄巾军。
　　张角领军攻打幽州地界，幽州太守连忙召校尉邹靖商议，邹靖说幽州兵少不能抵挡。建议写榜文到各县招募兵马。
　　榜文行到涿县，引出一名英雄，这人姓刘名备，字玄德。因家里贫寒，靠贩麻鞋、织草席为生。这天他进城来看榜文。


## 索引及存储

索引是通过embedding模型，得到文档nodes的嵌入向量，然后存储到向量数据库中，所以这步的目的是创建向量数据库

基于转换后的文档nodes，可以直接创建向量数据库

In [20]:
# 设置embedding model 
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url='http://192.168.3.155:11434')

# 构建向量数据库 
from llama_index.core import VectorStoreIndex
index=VectorStoreIndex(nodes=nodes,show_progress=True)
print(index)

Generating embeddings:   0%|          | 0/258 [00:00<?, ?it/s]

如果不需要定制转换方式，可以用以下接口快速从documents生成向量数据库

In [21]:
from llama_index.core import VectorStoreIndex

index=VectorStoreIndex.from_documents(documents=documents,show_progress=True)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/127 [00:00<?, ?it/s]

如果需要定制转换过程时，可以通过以下方式进行

In [24]:
from llama_index.core.node_parser import SentenceSplitter

text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=100)

# global
from llama_index.core import Settings
Settings.text_splitter = text_splitter

# per-index
index = VectorStoreIndex.from_documents(
    documents, transformations=[text_splitter],show_progress=True
)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/125 [00:00<?, ?it/s]

也有更加低级的接口，如果已知每个chunk的文本，可以直接定义文档的node对象，然后初始化向量数据库

In [25]:
from llama_index.core.schema import TextNode

node1 = TextNode(text="abc", id_="100")
node2 = TextNode(text="123", id_="200")

index = VectorStoreIndex([node1, node2],show_progress=True)

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

完成向量数据库的构建后，下一步可以将向量数据库持久化到磁盘，下次直接加载使用即可，而不必要重新构建

In [26]:
# 向量数据库持久化 
persist_dir='./vector_storage'
index.storage_context.persist(persist_dir=persist_dir)

# 加载持久化的向量数据库
from llama_index.core import StorageContext,load_index_from_storage
storage_context=StorageContext.from_defaults(persist_dir=persist_dir)
index=load_index_from_storage(storage_context=storage_context)

代码中的storage_context指的是不同的向量数据库，所谓不同，是指向量数据库存储机制、检索机制不同，如有的向量数据库擅长存储关系型数据，有的向量数据库可以进行更加复杂的检索方式。LlamaIndex支持多种向量数据库，比如chroma

In [29]:
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
# 需要先pip install llama-index-vector-stores-chroma 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

documents = SimpleDirectoryReader("data\三国演义白话文",recursive=True).load_data(show_progress=True)

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,show_progress=True
)

# create a query engine and query
query_engine = index.as_query_engine()
response = query_engine.query("张飞长坂坡做了什么事？")
print(response)

Loading files: 100%|█████████████| 41/41 [00:00<00:00, 694.39file/s]


Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/127 [00:00<?, ?it/s]

在长坂坡，张飞独立桥上，身后尘土飞扬。曹操怀疑这是诸葛亮的计策，不敢轻易靠近。张飞一声大喝，吓得曹军兵马人仰马翻而退。随后，他拆断了桥梁，并前去见刘备，报告了这一情况。
